<a href="https://colab.research.google.com/github/upinderpalsingh/5pisa/blob/main/nifty50_breakout_F%26O__Scan_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=3e4eea1c2b0a73c37e8bd6d60024aa49b2e01952c8cc4a6adc5f8c6eb1968321
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [3]:
!pip install yfinance ta

In [3]:
!pip install pandas numpy  ta  yfinance

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=07486ebfdf26fdc71c78cf2a3f9b1ce939d65d62b7f166c48a4e6b760ea293a7
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [4]:
pip install pandas ta requests

In [5]:
import requests
import pandas as pd
import ta
from datetime import datetime
import time

def fetch_nse_option_data(symbol="NIFTY", expiry="13-Feb-2025", strike_price=23700, option_type="CE"):
    """
    Fetches real-time NIFTY options data from NSE India
    """
    url = f"https://www.nseindia.com/api/option-chain-indices?symbol={symbol}"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept-Language": "en-US,en;q=0.9",
    }

    try:
        response = requests.get(url, headers=headers)
        data = response.json()

        # Extract option chain data
        option_chain = data["records"]["data"]
        df = pd.DataFrame(option_chain)

        # Print columns to debug missing data
        print("Fetched Data Columns:", df.columns)

        # Filter by expiry date & strike price
        df = df[
            (df["expiryDate"] == expiry) &
            (df["strikePrice"] == strike_price)
        ]

        if df.empty:
            print(f"No data found for {strike_price} strike and expiry {expiry}")
            return None

        if option_type not in ["CE", "PE"]:
            print("Invalid option type! Use 'CE' for Call or 'PE' for Put.")
            return None

        option_data = df.iloc[0][option_type]  # Extract Call/Put data
        return pd.DataFrame([option_data])  # Convert to DataFrame

    except Exception as e:
        print(f"Error fetching NSE data: {e}")
        return None

def calculate_indicators(df):
    """
    Calculate RSI, MACD, VWAP, EMA, and ADX indicators
    """
    if "lastPrice" not in df.columns:
        print("Error: 'lastPrice' column is missing in fetched data")
        return None

    df["LTP"] = df["lastPrice"]  # Last Traded Price as Close price
    df["VWAP"] = df["LTP"]  # Since NSE does not provide VWAP directly
    df["EMA9"] = ta.trend.ema_indicator(df["LTP"], window=9)
    df["EMA50"] = ta.trend.ema_indicator(df["LTP"], window=50)
    df["RSI"] = ta.momentum.RSIIndicator(df["LTP"], window=14).rsi()
    macd = ta.trend.MACD(df["LTP"])
    df["MACD"] = macd.macd()
    df["MACD_signal"] = macd.macd_signal()
    return df

def check_signals(df):
    """
    Identify Call or Put buy signals based on indicators
    """
    if df is None or df.empty:
        print("Error: Dataframe is empty or None. Cannot check signals.")
        return None, None, None

    latest = df.iloc[-1]
    signal = None
    entry_price = latest["LTP"]
    target_price = None

    if (latest["RSI"] > 60 and
        latest["LTP"] > latest["VWAP"] and
        latest["EMA9"] > latest["EMA50"] and
        latest["MACD"] > latest["MACD_signal"]):

        signal = "CALL BUY"
        target_price = entry_price * 1.05  # 5% target

    elif (latest["RSI"] < 40 and
          latest["LTP"] < latest["VWAP"] and
          latest["EMA9"] < latest["EMA50"] and
          latest["MACD"] < latest["MACD_signal"]):

        signal = "PUT BUY"
        target_price = entry_price * 0.95  # 5% target

    return signal, entry_price, target_price

def run_strategy_every_2_minutes():
    while True:
        option_data = fetch_nse_option_data(symbol="NIFTY", expiry="13-Feb-2025", strike_price=23700, option_type="PE")  # Change to "CE" for Call option

        if option_data is not None:
            df = calculate_indicators(option_data)

            if df is not None:
                # Check for signals
                signal, entry_price, target_price = check_signals(df)

                # Print recommendation
                if signal:
                    print(f"Signal: {signal}")
                    print(f"Entry Price: {entry_price:.2f}")
                    print(f"Target Price: {target_price:.2f}")
                else:
                    print("No clear trade signal found.")
            else:
                print("Failed to process indicators.")
        else:
            print("Could not fetch option data. Check NSE website manually.")

        time.sleep(120)  # Wait for 2 minutes before next iteration

# Run the strategy
run_strategy_every_2_minutes()

Fetched Data Columns: Index(['strikePrice', 'expiryDate', 'PE', 'CE'], dtype='object')
No clear trade signal found.
Fetched Data Columns: Index(['strikePrice', 'expiryDate', 'PE', 'CE'], dtype='object')
No clear trade signal found.
Fetched Data Columns: Index(['strikePrice', 'expiryDate', 'PE', 'CE'], dtype='object')
No clear trade signal found.
Fetched Data Columns: Index(['strikePrice', 'expiryDate', 'PE', 'CE'], dtype='object')
No clear trade signal found.
Fetched Data Columns: Index(['strikePrice', 'expiryDate', 'PE', 'CE'], dtype='object')
No clear trade signal found.
Fetched Data Columns: Index(['strikePrice', 'expiryDate', 'PE', 'CE'], dtype='object')
No clear trade signal found.
Fetched Data Columns: Index(['strikePrice', 'expiryDate', 'PE', 'CE'], dtype='object')
No clear trade signal found.
Fetched Data Columns: Index(['strikePrice', 'expiryDate', 'PE', 'CE'], dtype='object')
No clear trade signal found.
Fetched Data Columns: Index(['strikePrice', 'expiryDate', 'PE', 'CE'], d

KeyboardInterrupt: 